In [41]:
import numpy as np
import matplotlib.pyplot as plt

from platform import python_version
print('python_version = ' + python_version())

python_version = 3.6.9


In [42]:
print('tf version = ' + tf.__version__)

tf version = 1.14.0


In [43]:
import cv2 # pip install opencv-python
import os 
import random
cwd = os.getcwd()
print('current working dir ' + cwd)

current working dir /Users/rahal/Development/Anaconda/Flowers


In [44]:
CATEGORIES = ['daisy', 'dandelion', 'rose', 'sunflower', 'tulip']
TRAINING_DATA = []
IMAGE_SIZE = 128

def resize_image(img):
    resized_img = cv2.resize(img, (IMAGE_SIZE, IMAGE_SIZE))
    # plt.imshow(resized_img, cmap='gray')
    # plt.show()
    return resized_img

def show_image(img, name):
    print('Name = ' + name)
    plt.imshow(img, cmap='gray')
    plt.show()

def create_training_data():
    for category in CATEGORIES:
        path = cwd + '/' + category
        print(path)
        class_num = CATEGORIES.index(category)
        for img_name in os.listdir(path):
            try:
                img = cv2.imread(path + '/' + img_name, cv2.IMREAD_GRAYSCALE)
                resized_img = resize_image(img)
                # show_image(resized_img, img_name)
                # img is just a 2D or 3D array, depending on gray scale or not
                TRAINING_DATA.append([resized_img, class_num])
            except Exception as e:
                # pass
                print('Exception at ' + img_name)
                print(e)


create_training_data()    
print('TRAINING_DATA array length = ' + str(len(TRAINING_DATA)))

# Shuffel training data array
random.shuffle(TRAINING_DATA)

/Users/rahal/Development/Anaconda/Flowers/daisy
/Users/rahal/Development/Anaconda/Flowers/dandelion
/Users/rahal/Development/Anaconda/Flowers/rose
/Users/rahal/Development/Anaconda/Flowers/sunflower
/Users/rahal/Development/Anaconda/Flowers/tulip
TRAINING_DATA array length = 4323


In [45]:
# for sample in TRAINING_DATA:
#     print(CATEGORIES[sample[1]])

In [46]:
# Prep to feed TRAINING_DATA to our neural network

train_images_X = [] # features
train_labels_y = [] # labels

for features, label in TRAINING_DATA:
    train_images_X.append(features)
    train_labels_y.append(label)
    
# train_images_X have to be a numpy array
train_images_X = np.array(train_images_X).reshape(-1, IMAGE_SIZE, IMAGE_SIZE, 1)  # 1 cuz of gray scale


In [47]:
# save the data

np.save('train_images_X.npy', train_images_X)
np.save('train_labels_y.npy', train_labels_y)


In [48]:

# print(train_images_X.shape[1:])
# print(train_images_X.shape)
# print(train_images_X[1])


In [54]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D

from tensorflow.keras.utils import to_categorical

# load the data
X = np.load('train_images_X.npy')
y = np.load('train_labels_y.npy')

# Normalize data
# Easiset way to normalize data is to scale that data
# For images we know min=0 and max=255

# Keras does not use integer labels for the usual crossentropy loss, instead it expects a binary vector 
# (called "one-hot"), where the vector is just 0's and a 1 over the index of the right class.
# You can easily convert your labels to this format with to_categorical

train_images_X = X/255.0
train_labels_y = to_categorical(y)

print(train_images_X.shape)
print(train_labels_y.shape)

model = Sequential() # simple sequential model

# train_images_shape = ''.join(train_images_X.shape[1:])
# print('Input Shape: ' + train_images_shape)
print(train_images_X.shape[1:])

# Generate a 2 x 64 layered converlutional neural network

# Layer 1 with input
model.add(Conv2D(64, (3,3), input_shape=train_images_X.shape[1:])) # Add the convolutional layer
model.add(Activation('relu')) # Rectified Linear Unit
model.add(MaxPooling2D(pool_size=(2,2)))

# Layer 2
model.add(Conv2D(64, (3,3))) # Add the convolutional layer
model.add(Activation('relu')) # Rectified Linear Unit
model.add(MaxPooling2D(pool_size=(2,2)))

# 64 node final dense layer
model.add(Flatten()) # Flatten because CNN is 2d and Dense layer is 1D
model.add(Dense(64))
model.add(Activation('relu'))
          
model.add(Dense(5))
model.add(Activation('sigmoid'))

# compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(train_images_X, 
          train_labels_y, 
          batch_size=32,
          epochs=3,
          validation_split=0.1)


# Conv2D - https://keras.io/layers/convolutional/
# Activations - https://keras.io/activations/
# MaxPooling2D - https://keras.io/layers/pooling/#maxpooling2dMaxPooling2D - 


(4323, 128, 128, 1)
(4323, 5)
(128, 128, 1)
Train on 3890 samples, validate on 433 samples
Epoch 1/3
3890/3890 [==============================] - 116s 30ms/sample - loss: 0.4864 - acc: 0.7991 - val_loss: 0.4800 - val_acc: 0.8005
Epoch 2/3
3890/3890 [==============================] - 125s 32ms/sample - loss: 0.4268 - acc: 0.8187 - val_loss: 0.4217 - val_acc: 0.8263
Epoch 3/3
3890/3890 [==============================] - 173s 44ms/sample - loss: 0.3617 - acc: 0.8470 - val_loss: 0.4225 - val_acc: 0.8245


In [55]:
# save the model
# Keras separates the concerns of saving your model architecture and saving your model weights.

# Model weights are saved to HDF5 format. 
# This is a grid format that is ideal for storing multi-dimensional arrays of numbers

# The model structure can be described and saved using two different formats: JSON and YAML.

# model.json
# model.h5

NAME = 'flowers-cnn-64x2-model'
MODEL_NAME_JSON = NAME + '.json'
WEIGHTS_NAME_H5 = NAME + '.h5'

# serialize model to JSON
model_json = model.to_json()
with open(MODEL_NAME_JSON, "w") as json_file:
    json_file.write(model_json)

# serialize weights to HDF5
model.save_weights(WEIGHTS_NAME_H5)

print("Saved model to disk")

Saved model to disk


In [37]:
# Load Model
# from tensorflow.keras.models import model_from_json

# # load json and create model
# json_file = open(MODEL_NAME_JSON, 'r')
# loaded_model_json = json_file.read()
# json_file.close()
# loaded_model = model_from_json(loaded_model_json)

## load weights into new model
# loaded_model.load_weights(WEIGHTS_NAME_H5)
# print("Loaded model from disk")

## Compile loaded model
# loaded_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
